In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from kaggle_datasets import KaggleDatasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import PIL
import shutil
import csv

import os
os.system('pip install /kaggle/input/kerasapplications -q')
os.system('pip install /kaggle/input/efficientnetkeras/ -q --no-deps')

import efficientnet.tfkeras as efn

In [ ]:
HEIGHT,WIDTH = 512,512
CHANNELS = 3
BATCH_SIZE = 32
SEED = 1

In [ ]:
import tensorflow_addons as tfa

def compile_model(model, lr=0.001):
    
    optimizer = tf.keras.optimizers.Adam(lr=lr)
    
    loss = tf.keras.losses.BinaryCrossentropy()
        
    metrics = [
       tfa.metrics.F1Score(num_classes = NUM_CLASSES,average = "macro", name = "f1_score"),
       tf.keras.metrics.BinaryAccuracy(name='acc')
    ]

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model

In [ ]:
METRIC = "val_f1_score"

def create_callbacks(metric = METRIC):
    
    cpk_path = './best_model.h5'
    
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath=cpk_path,
        monitor= metric,
        mode='max',
        save_best_only=True,
        verbose=1,
    )

    reducelr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor= metric,
        mode='max',
        factor=0.1,
        patience=3,
        verbose=0
    )

    earlystop = tf.keras.callbacks.EarlyStopping(
        monitor= metric,
        mode='max',
        patience=10, 
        verbose=1
    )
    
    callbacks = [checkpoint, reducelr, earlystop]         
    
    return callbacks

In [ ]:
from keras.models import load_model

In [ ]:
# Public Score = 0.59
# model = load_model("../input/mymodel/mymodel.h5")

model = load_model("../input/mymodel/mymodel.h5")

In [ ]:
names = os.listdir('../input/plant-pathology-2021-fgvc8/test_images/')

In [ ]:
def _parse(name):
#     with open('../input/plant-pathology-2021-fgvc8/test_images/'+ name, "rb") as local_file:
#         image_string = local_file.read()
    image_string = tf.io.read_file('../input/plant-pathology-2021-fgvc8/test_images/' + name)
    image_decoded = tf.image.decode_jpeg(image_string)
    imgs = tf.image.resize(image_decoded, [WIDTH, HEIGHT])
    return imgs/255



dataset = tf.data.Dataset.from_tensor_slices((tf.constant(names)))\
                               .map(_parse, num_parallel_calls=tf.data.AUTOTUNE)\
                               .batch(32)\
                               .prefetch(tf.data.AUTOTUNE)
print(dataset)

In [ ]:
y_pred = model.predict(dataset, verbose=1)

In [ ]:
label_names=['scab','frog_eye_leaf_spot','rust','complex','powdery_mildew']

y = np.around(y_pred)
i = 0
labels = []
for i in range(len(y)):
    check = 0
    vec = str()
    for j in range(len(label_names)):
        if(y[i][j]==1):
            check+=1
            vec = vec + label_names[j] + " "
    if(check==0):
        vec = vec + 'healthy'
    labels.append(vec)

In [ ]:
df = pd.DataFrame({'image':names, 'labels':labels})

In [ ]:
df.head()

In [ ]:
df.to_csv('./submission.csv', index=False)